In [23]:
from sched import scheduler

import availability_parser
import session_generator
import scheduler
from input import user_preferences, availability, courses, assignments

In [24]:
# Build session data
study_sessions = session_generator.build_study_sessions(courses, user_preferences)
assignment_sessions = session_generator.build_assignment_sessions(assignments, user_preferences)

# Print study sessions
print("📚 Study Sessions for Courses:")

# Group by course → topic → sessions
from collections import defaultdict

course_topic_sessions = defaultdict(lambda: defaultdict(list))
for s in study_sessions:
    course_topic_sessions[s["course"]][s["topic"]].append(s)

for course_name, topics in course_topic_sessions.items():
    print(f"\n📘 Course: {course_name}")
    for topic, sessions in topics.items():
        total = sum(s["duration"] for s in sessions)
        session_info = ", ".join(f"{s['duration']} min" for s in sessions)
        print(f"  • {topic} → Total Adjusted: {total} min → Sessions: {session_info}")

# Print assignment sessions
print("\n" + "=" * 30 + "\n")
print("📝 Assignment Sessions:")

grouped_assignments = {}
for s in assignment_sessions:
    key = (s["course"], s["title"])
    grouped_assignments.setdefault(key, []).append(s)

for (course_name, title), sessions in grouped_assignments.items():
    total = sum(s["duration"] for s in sessions)
    session_info = ", ".join(f"{s['duration']} min" for s in sessions)
    print(f"\n📌 Assignment: {title} ({course_name})")
    print(f"  • Estimated Time: {total} min → Sessions: {session_info}")

📚 Study Sessions for Courses:

📘 Course: Operating Systems
  • CPU Scheduling → Total Adjusted: 160 min → Sessions: 30 min, 50 min, 50 min, 30 min
  • Deadlocks → Total Adjusted: 140 min → Sessions: 30 min, 80 min, 30 min
  • Virtual Memory → Total Adjusted: 230 min → Sessions: 30 min, 80 min, 80 min, 40 min

📘 Course: Data Structures and Algorithms
  • Trees & Graphs → Total Adjusted: 150 min → Sessions: 30 min, 90 min, 30 min
  • Sorting Algorithms → Total Adjusted: 100 min → Sessions: 30 min, 40 min, 30 min
  • Dynamic Programming → Total Adjusted: 200 min → Sessions: 30 min, 70 min, 70 min, 30 min

📘 Course: Machine Learning
  • Regression Models → Total Adjusted: 130 min → Sessions: 30 min, 70 min, 30 min
  • Classification → Total Adjusted: 180 min → Sessions: 30 min, 60 min, 60 min, 30 min
  • Neural Networks → Total Adjusted: 250 min → Sessions: 30 min, 90 min, 90 min, 40 min

📘 Course: Database Systems
  • SQL Joins → Total Adjusted: 90 min → Sessions: 30 min, 30 min, 30 min
 

In [25]:
slots = availability_parser.generate_slots(availability, user_preferences)
print("Generated Slots:")
for slot in slots:
    print(f"ID {slot['id']} | {slot['date']} | {slot['start']}–{slot['end']} ({slot['duration']} min)")

Generated Slots:
ID 0 | 11/02/2025 | 10:00–11:30 (90 min)
ID 1 | 11/02/2025 | 14:00–15:30 (90 min)
ID 2 | 11/02/2025 | 15:45–17:00 (75 min)
ID 3 | 11/02/2025 | 20:00–21:30 (90 min)
ID 4 | 12/02/2025 | 09:00–10:30 (90 min)
ID 5 | 12/02/2025 | 14:00–15:30 (90 min)
ID 6 | 12/02/2025 | 20:00–21:30 (90 min)
ID 7 | 13/02/2025 | 10:00–11:30 (90 min)
ID 8 | 13/02/2025 | 14:00–15:30 (90 min)
ID 9 | 13/02/2025 | 15:45–17:00 (75 min)
ID 10 | 13/02/2025 | 20:00–21:30 (90 min)
ID 11 | 14/02/2025 | 10:00–11:30 (90 min)
ID 12 | 14/02/2025 | 14:00–15:30 (90 min)
ID 13 | 14/02/2025 | 15:45–17:15 (90 min)
ID 14 | 14/02/2025 | 17:30–18:00 (30 min)
ID 15 | 14/02/2025 | 20:00–21:30 (90 min)
ID 16 | 15/02/2025 | 08:00–09:30 (90 min)
ID 17 | 15/02/2025 | 09:45–11:00 (75 min)
ID 18 | 15/02/2025 | 14:00–15:30 (90 min)
ID 19 | 15/02/2025 | 20:00–21:30 (90 min)
ID 20 | 17/02/2025 | 10:00–11:30 (90 min)
ID 21 | 17/02/2025 | 14:00–15:30 (90 min)
ID 22 | 17/02/2025 | 15:45–17:00 (75 min)
ID 23 | 17/02/2025 | 20:00–

In [26]:
slots = availability_parser.generate_slots(availability, user_preferences)
slot_availability = {slot["id"]: slot["duration"] for slot in slots}
reserved_reviews = scheduler.schedule_review_sessions(study_sessions, slots, slot_availability)

print("\n🗓️ Reserved Review Slots:")
if reserved_reviews:
    for review in reserved_reviews:
        print(f"📌 {review['course']} → {review['topic']} → Review on {review['date']} from {review['start']} to {review['end']} ({review['duration']} min)")
else:
    print("⚠️ No review slots were reserved.")


🗓️ Reserved Review Slots:
📌 Operating Systems → CPU Scheduling → Review on 24/02/2025 from 09:00 to 10:30 (30 min)
📌 Operating Systems → Deadlocks → Review on 24/02/2025 from 14:00 to 15:30 (30 min)
📌 Operating Systems → Virtual Memory → Review on 24/02/2025 from 20:00 to 21:30 (40 min)
📌 Data Structures and Algorithms → Trees & Graphs → Review on 21/02/2025 from 09:00 to 10:30 (30 min)
📌 Data Structures and Algorithms → Sorting Algorithms → Review on 21/02/2025 from 14:00 to 15:30 (30 min)
📌 Data Structures and Algorithms → Dynamic Programming → Review on 21/02/2025 from 20:00 to 21:30 (30 min)
📌 Machine Learning → Regression Models → Review on 22/02/2025 from 09:00 to 10:30 (30 min)
📌 Machine Learning → Classification → Review on 22/02/2025 from 14:00 to 15:30 (30 min)
📌 Machine Learning → Neural Networks → Review on 22/02/2025 from 20:00 to 21:30 (40 min)
📌 Database Systems → SQL Joins → Review on 26/02/2025 from 09:00 to 10:30 (30 min)
📌 Database Systems → Normalization → Review o

In [27]:
scheduled_related = scheduler.schedule_related_topic_sessions(assignments, study_sessions, slots, slot_availability)

# Print what was scheduled
print("\n📌 Study Sessions Related to Assignments:")
if scheduled_related:
    for s in scheduled_related:
        print(f"📚 {s['course']} → {s['topic']} on {s['date']} from {s['start']} to {s['end']} ({s['duration']} min)")
else:
    print("⚠️ No related study sessions were scheduled.")



📌 Study Sessions Related to Assignments:
📚 Computer Networks → TCP/IP Model on 11/02/2025 from 10:00 to 11:30 (30 min)
📚 Computer Networks → TCP/IP Model on 11/02/2025 from 14:00 to 15:30 (40 min)
📚 Machine Learning → Regression Models on 11/02/2025 from 20:00 to 21:30 (30 min)
📚 Machine Learning → Regression Models on 12/02/2025 from 09:00 to 10:30 (70 min)
📚 Machine Learning → Classification on 12/02/2025 from 14:00 to 15:30 (30 min)
📚 Machine Learning → Classification on 12/02/2025 from 20:00 to 21:30 (60 min)
📚 Machine Learning → Classification on 13/02/2025 from 10:00 to 11:30 (60 min)
📚 Machine Learning → Neural Networks on 13/02/2025 from 14:00 to 15:30 (30 min)
📚 Machine Learning → Neural Networks on 13/02/2025 from 20:00 to 21:30 (90 min)
📚 Machine Learning → Neural Networks on 14/02/2025 from 10:00 to 11:30 (90 min)


## Scheduling

In [28]:
# Build session data
study_sessions = session_generator.build_study_sessions(courses, user_preferences)
assignment_sessions = session_generator.build_assignment_sessions(assignments, user_preferences)

# Shared state
slots = availability_parser.generate_slots(availability, user_preferences)
slot_availability = {slot["id"]: slot["duration"] for slot in slots}

# STEP 1: Reserve review sessions
reserved_reviews = scheduler.schedule_review_sessions(study_sessions, slots, slot_availability)

# STEP 2: Schedule study sessions related to assignments
scheduled_related = scheduler.schedule_related_topic_sessions(assignments, study_sessions, slots, slot_availability)

# STEP 3: Schedule assignment sessions
scheduled_assignments = scheduler.schedule_assignment_sessions(assignments, assignment_sessions, slots, slot_availability)

# OUTPUTS

# Reserved review sessions
print("\n🗓️ Reserved Review Slots:")
if reserved_reviews:
    for review in reserved_reviews:
        print(f"📌 {review['course']} → {review['topic']} → Review on {review['date']} from {review['start']} to {review['end']} ({review['duration']} min)")
else:
    print("⚠️ No review slots were reserved.")

# Study sessions linked to assignments
print("\n📌 Study Sessions Related to Assignments:")
if scheduled_related:
    for s in scheduled_related:
        print(f"📚 {s['course']} → {s['topic']} on {s['date']} from {s['start']} to {s['end']} ({s['duration']} min)")
else:
    print("⚠️ No related study sessions were scheduled.")

# ✅ Assignment sessions
print("\n📝 Scheduled Assignment Sessions:")
if scheduled_assignments:
    for a in scheduled_assignments:
        print(f"📌 {a['course']} → {a['title']} on {a['date']} from {a['start']} to {a['end']} ({a['duration']} min)")
else:
    print("⚠️ No assignment sessions were scheduled.")



🗓️ Reserved Review Slots:
📌 Operating Systems → CPU Scheduling → Review on 24/02/2025 from 09:00 to 10:30 (30 min)
📌 Operating Systems → Deadlocks → Review on 24/02/2025 from 14:00 to 15:30 (30 min)
📌 Operating Systems → Virtual Memory → Review on 24/02/2025 from 20:00 to 21:30 (40 min)
📌 Data Structures and Algorithms → Trees & Graphs → Review on 21/02/2025 from 09:00 to 10:30 (30 min)
📌 Data Structures and Algorithms → Sorting Algorithms → Review on 21/02/2025 from 14:00 to 15:30 (30 min)
📌 Data Structures and Algorithms → Dynamic Programming → Review on 21/02/2025 from 20:00 to 21:30 (30 min)
📌 Machine Learning → Regression Models → Review on 22/02/2025 from 09:00 to 10:30 (30 min)
📌 Machine Learning → Classification → Review on 22/02/2025 from 14:00 to 15:30 (30 min)
📌 Machine Learning → Neural Networks → Review on 22/02/2025 from 20:00 to 21:30 (40 min)
📌 Database Systems → SQL Joins → Review on 26/02/2025 from 09:00 to 10:30 (30 min)
📌 Database Systems → Normalization → Review o